### Semi- Super Vised Node classification using traditional machine learning algorithms






In [ ]:
#!pip install dgl
#!pip install node2vec

import time

import numpy as np                         # load all neccesaary libraries 
import matplotlib.pyplot as plt
import pandas as pd
                                                
import scipy as sp
import scipy.sparse.linalg as linalg
import scipy.cluster.hierarchy as hr
from scipy.spatial.distance import pdist, squareform


import sklearn.metrics as metrics
import sklearn.utils as utils
import sklearn.linear_model as linear_model
import sklearn.svm as svm
import sklearn.cluster as cluster
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler


import networkx as nx
from node2vec import Node2Vec

import seaborn as sns
from dgl.nn.pytorch import GraphConv
import torch.nn as nn
import torch.nn.functional as F

Read the graph data or adjacency list from file and form a basic networkx graph

In [ ]:
G = nx.read_adjlist('adjedges.txt', create_using = nx.DiGraph(), nodetype = int)
        ## create graph from edge list
#G.nodes()
#G.edges()              # no of nodes
G.number_of_nodes()

In [ ]:
#G.nodes()
#G.edges()              # no of nodes
G.number_of_nodes()

36928

#### Node Emebeddings using node2vec 
By samplig startegy nodes are emebedded from high dimension to low dimension vectors

In [ ]:
#pre-compute the probabilities and generate walks :
node2vec = Node2Vec(G, dimensions=64, walk_length=20, num_walks=200, workers=4)
#embed the nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)


Computing transition probabilities: 100%|██████████| 36928/36928 [00:01<00:00, 23271.54it/s]


In [ ]:
#model.wv.get_vector('12828558')               # get inidividual node feature

array([-4.83293133e-03,  3.32037499e-03, -5.13896067e-03,  6.01090537e-03,
       -5.84459282e-04, -2.91788671e-03, -3.00790160e-03, -5.93654346e-03,
       -4.02862858e-03,  5.56175970e-03, -5.90341398e-03, -5.54963108e-03,
       -5.59177203e-03, -4.70674504e-03,  3.53438267e-03,  7.02380156e-03,
       -4.72784694e-03,  6.11006806e-04,  5.34979627e-03,  7.75020989e-03,
       -3.38701555e-03, -6.27190201e-03,  6.41502964e-04,  6.55236514e-03,
        1.72545388e-03,  1.86560873e-03, -7.07889965e-04, -2.49871030e-03,
       -5.55223692e-03, -6.08281884e-03, -4.32439987e-03, -3.76389083e-03,
        5.27886162e-03, -5.34010760e-04, -3.77670210e-03, -2.63359328e-03,
        7.72494168e-05, -4.12104093e-03, -4.06824891e-03,  5.26920985e-03,
        1.05634375e-04,  5.99916512e-03,  2.34934990e-03,  7.48800952e-03,
        2.01709289e-03,  3.70375987e-04,  5.61880786e-03,  4.20704298e-03,
       -5.74026443e-03, -2.48439028e-03, -2.69574055e-04,  2.89472146e-03,
       -3.05547705e-03,  

#### Loading training data
Now we load training data where a list of nodes along with their class label is given

In [ ]:
data_train = pd.read_csv('labels.txt', sep=" ", header=None)     # create training data with column names
data_train.columns=["node_id","class"]                           

In [ ]:
data_train

,node_id,class
0,12828558,0
1,66779408,0
2,38902949,0
3,33450563,0
4,57470294,0
...,...,...
18715,39790331,4
18716,47870669,4
18717,28485685,4
18718,48611755,4


#### Now we add a column of node embeddings to this dataframe along side each node_id and these are node features . We do so by applying fucntion row wise

In [ ]:
data_train['node_embeddings'] = data_train.apply(lambda row: model.wv.get_vector(str(row.node_id)) , axis = 1)

###### We know use node embeddings of nodes as Xtrain  and labels for respective nodes as Ytrain

In [ ]:
Xtrain = data_train['node_embeddings'].values                   # get x train and ytrain values where 
Ytrain = data_train['class'].values
#Xtrain

array([array([-4.83293133e-03,  3.32037499e-03, -5.13896067e-03,  6.01090537e-03,
       -5.84459282e-04, -2.91788671e-03, -3.00790160e-03, -5.93654346e-03,
       -4.02862858e-03,  5.56175970e-03, -5.90341398e-03, -5.54963108e-03,
       -5.59177203e-03, -4.70674504e-03,  3.53438267e-03,  7.02380156e-03,
       -4.72784694e-03,  6.11006806e-04,  5.34979627e-03,  7.75020989e-03,
       -3.38701555e-03, -6.27190201e-03,  6.41502964e-04,  6.55236514e-03,
        1.72545388e-03,  1.86560873e-03, -7.07889965e-04, -2.49871030e-03,
       -5.55223692e-03, -6.08281884e-03, -4.32439987e-03, -3.76389083e-03,
        5.27886162e-03, -5.34010760e-04, -3.77670210e-03, -2.63359328e-03,
        7.72494168e-05, -4.12104093e-03, -4.06824891e-03,  5.26920985e-03,
        1.05634375e-04,  5.99916512e-03,  2.34934990e-03,  7.48800952e-03,
        2.01709289e-03,  3.70375987e-04,  5.61880786e-03,  4.20704298e-03,
       -5.74026443e-03, -2.48439028e-03, -2.69574055e-04,  2.89472146e-03,
       -3.05547705

##### Xtrain values may not be of same length and hence padding is done to maintain same length for each node embedding  .
We use keras function of pad sequences 

In [ ]:
from keras.preprocessing.sequence import pad_sequences
Xtrain = pad_sequences(Xtrain)

###### Splitting into train and test sets using Skicit-learn

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(Xtrain, Ytrain, test_size = 0.75, random_state = 42)

##### Now data is trained for various models and accuracy is predicted 

##### Multiclass classifications models
###### 1)  Random forest

In [ ]:
# Import the model we are using                                   
from sklearn.ensemble import RandomForestClassifier
classifier_1= RandomForestClassifier(n_estimators = 1000,random_state = 42)
classifier_1.fit(list(train_features),train_labels )

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
# Predicting the Test set results
y_pred = classifier_1.predict(test_features)
from sklearn import metrics
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))

Accuracy: 0.3782763532763533


###### 2)  Support vector classifier

In [ ]:
from sklearn.svm import SVC 

from sklearn.metrics import confusion_matrix

In [ ]:
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(Xtrain, Ytrain) 
svm_predictions = svm_model_linear.predict(test_features)
# model accuracy for X_test   
accuracy = svm_model_linear.score(test_features, test_labels) 
  
# creating a confusion matrix 
cm = confusion_matrix(test_labels, svm_predictions) 
accuracy

0.3764957264957265

###### 3)  Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(Xtrain, Ytrain) 
dtree_predictions = dtree_model.predict(test_features)

In [ ]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(test_labels, dtree_predictions))

Accuracy: 0.36217948717948717


###### 2)  Naive Bayes classifier 

In [ ]:
# training a Naive Bayes classifier 
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(Xtrain, Ytrain) 
gnb_predictions = gnb.predict(test_features) 
  
# accuracy on X_test 
accuracy = gnb.score(test_features, test_labels)
print(accuracy)

0.3492877492877493
